In [75]:
# Import libraries
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import SplineTransformer
import datetime
from sklearn.cluster import KMeans
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")
%matplotlib inline
sns.set(rc={'figure.figsize':(12,3)})
pd.set_option('display.max_columns', 50)
from sklearn.decomposition import PCA, FastICA, TruncatedSVD
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error,make_scorer
from sklearn.model_selection import train_test_split, KFold,cross_validate
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder
le=LabelEncoder()
#pd.options.display.max_rows=None
plt.style.use("seaborn-whitegrid")
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm_notebook
X_test= pd.read_csv('IEEE Hackathon/Data/1 - Savanna Preserve/1_X_test.csv')
X_train= pd.read_csv('IEEE Hackathon/Data/1 - Savanna Preserve/1_X_train.csv')
y_test= pd.read_csv('IEEE Hackathon/Data/1 - Savanna Preserve/1_y_test.csv')
y_train= pd.read_csv('IEEE Hackathon/Data/1 - Savanna Preserve/1_y_test.csv')
ss = pd.read_csv('IEEE Hackathon/Data/1 - Savanna Preserve/sample submission/1_sample_submission.csv')

train= pd.concat([y_train,y_test])
test= pd.concat([ X_test,X_test])

In [76]:
ss

,Unnamed: 0,location_id,date,temperature_2m,relative_humidity_2m
0,0,1,2024-09-24 00:00:00+00:00,NaN,NaN
1,1,1,2024-09-24 01:00:00+00:00,NaN,NaN
2,2,1,2024-09-24 02:00:00+00:00,NaN,NaN
3,3,1,2024-09-24 03:00:00+00:00,NaN,NaN
4,4,1,2024-09-24 04:00:00+00:00,NaN,NaN
...,...,...,...,...,...
163,163,1,2024-09-30 19:00:00+00:00,NaN,NaN
164,164,1,2024-09-30 20:00:00+00:00,NaN,NaN
165,165,1,2024-09-30 21:00:00+00:00,NaN,NaN
166,166,1,2024-09-30 22:00:00+00:00,NaN,NaN


In [77]:
train

,Unnamed: 0,location_id,date,temperature_2m,relative_humidity_2m
0,0,1,2024-09-24 00:00:00+00:00,15.547000,70.0
1,1,1,2024-09-24 01:00:00+00:00,15.047000,69.0
2,2,1,2024-09-24 02:00:00+00:00,14.646999,68.0
3,3,1,2024-09-24 03:00:00+00:00,14.447000,70.0
4,4,1,2024-09-24 04:00:00+00:00,14.497000,75.0
...,...,...,...,...,...
163,163,1,2024-09-30 19:00:00+00:00,19.046999,91.0
164,164,1,2024-09-30 20:00:00+00:00,18.697000,92.0
165,165,1,2024-09-30 21:00:00+00:00,18.497000,94.0
166,166,1,2024-09-30 22:00:00+00:00,18.397000,94.0


In [78]:
test

,Unnamed: 0,location_id,date,temperature_2m_previous_day1,temperature_2m_previous_day2,temperature_2m_previous_day3,temperature_2m_previous_day4,temperature_2m_previous_day5,temperature_2m_previous_day6,temperature_2m_previous_day7,relative_humidity_2m_previous_day1,relative_humidity_2m_previous_day2,relative_humidity_2m_previous_day3,relative_humidity_2m_previous_day4,relative_humidity_2m_previous_day5,relative_humidity_2m_previous_day6,relative_humidity_2m_previous_day7
0,0,1,2024-09-24 00:00:00+00:00,15.597000,17.247000,16.796999,17.046999,16.247000,16.197000,16.195500,62.0,57.0,62.0,63.0,68.0,71.0,68.0
1,1,1,2024-09-24 01:00:00+00:00,15.396999,16.497000,16.397000,16.597000,15.747000,15.447000,15.345500,65.0,64.0,68.0,66.0,71.0,76.0,73.0
2,2,1,2024-09-24 02:00:00+00:00,15.297000,16.047000,16.047000,16.197000,15.297000,14.646999,14.695499,69.0,70.0,72.0,70.0,74.0,81.0,75.0
3,3,1,2024-09-24 03:00:00+00:00,15.197000,15.896999,15.896999,16.347000,15.447000,14.646999,14.995500,73.0,73.0,73.0,71.0,75.0,82.0,74.0
4,4,1,2024-09-24 04:00:00+00:00,15.197000,16.097000,15.997000,17.247000,16.597000,15.947000,16.745500,75.0,72.0,74.0,70.0,72.0,78.0,66.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,163,1,2024-09-30 19:00:00+00:00,17.497000,19.647000,18.347000,19.147000,18.447000,19.247000,19.495500,84.0,91.0,86.0,90.0,84.0,90.0,78.0
164,164,1,2024-09-30 20:00:00+00:00,17.546999,19.247000,17.997000,19.046999,18.247000,18.697000,19.645500,85.0,92.0,88.0,89.0,86.0,93.0,76.0
165,165,1,2024-09-30 21:00:00+00:00,17.247000,19.046999,17.597000,18.847000,18.046999,18.147000,19.595499,87.0,92.0,90.0,89.0,89.0,95.0,75.0
166,166,1,2024-09-30 22:00:00+00:00,16.897000,18.847000,17.147000,18.497000,17.747000,17.697000,19.195500,89.0,93.0,91.0,91.0,92.0,96.0,75.0


In [79]:
ss

,Unnamed: 0,location_id,date,temperature_2m,relative_humidity_2m
0,0,1,2024-09-24 00:00:00+00:00,NaN,NaN
1,1,1,2024-09-24 01:00:00+00:00,NaN,NaN
2,2,1,2024-09-24 02:00:00+00:00,NaN,NaN
3,3,1,2024-09-24 03:00:00+00:00,NaN,NaN
4,4,1,2024-09-24 04:00:00+00:00,NaN,NaN
...,...,...,...,...,...
163,163,1,2024-09-30 19:00:00+00:00,NaN,NaN
164,164,1,2024-09-30 20:00:00+00:00,NaN,NaN
165,165,1,2024-09-30 21:00:00+00:00,NaN,NaN
166,166,1,2024-09-30 22:00:00+00:00,NaN,NaN


In [80]:
def date(df):
    df['date']=pd.to_datetime(df['date'])
    df['month'] = pd.to_datetime(df['date']).dt.month
    df['weekofyear'] = pd.to_datetime(df['date']).dt.weekofyear
    df['day'] = pd.to_datetime(df['date']).dt.day
    df['sin_day'] = np.sin(2 * np.pi * df['day'] / 30)
    df['sin_weekofyear'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)
    return df
train=date(train)
test=date(test)
X_test=date(X_test)

In [81]:
for i in range(1, 8):
    train[f'temperature_2m_previous_day{i}'] = train['temperature_2m'].shift(i)
    train[f'relative_humidity_2m_previous_day{i}'] = train['relative_humidity_2m'].shift(i)

In [82]:
train.shape,test.shape

((336, 25), (336, 23))

In [83]:
train.columns

Index(['Unnamed: 0', 'location_id', 'date', 'temperature_2m',
       'relative_humidity_2m', 'month', 'weekofyear', 'day', 'sin_day',
       'sin_weekofyear', 'sin_month', 'temperature_2m_previous_day1',
       'relative_humidity_2m_previous_day1', 'temperature_2m_previous_day2',
       'relative_humidity_2m_previous_day2', 'temperature_2m_previous_day3',
       'relative_humidity_2m_previous_day3', 'temperature_2m_previous_day4',
       'relative_humidity_2m_previous_day4', 'temperature_2m_previous_day5',
       'relative_humidity_2m_previous_day5', 'temperature_2m_previous_day6',
       'relative_humidity_2m_previous_day6', 'temperature_2m_previous_day7',
       'relative_humidity_2m_previous_day7'],
      dtype='object')

In [84]:
col=train.drop(['date','temperature_2m','Unnamed: 0','relative_humidity_2m','location_id'],axis=1).columns
X=train[col]
X_=X_test[col]
t=train.temperature_2m


In [85]:
col1=train.drop(['date','relative_humidity_2m','temperature_2m','Unnamed: 0','location_id'],axis=1).columns
X1=train[col1]
X1_=X_test[col1]
r=train.relative_humidity_2m

In [86]:
ctb = CatBoostRegressor().fit(X,t)
xgr = XGBRegressor().fit(X,t)
lgr = LGBMRegressor().fit(X,t)

Learning rate set to 0.034462
0:	learn: 4.0557167	total: 3.32ms	remaining: 3.32s
1:	learn: 3.9527745	total: 5.77ms	remaining: 2.88s
2:	learn: 3.8562725	total: 8.06ms	remaining: 2.68s
3:	learn: 3.7651051	total: 10.5ms	remaining: 2.61s
4:	learn: 3.6729626	total: 13.6ms	remaining: 2.7s
5:	learn: 3.5762838	total: 16.1ms	remaining: 2.67s
6:	learn: 3.4861685	total: 18.5ms	remaining: 2.62s
7:	learn: 3.4007955	total: 20.8ms	remaining: 2.58s
8:	learn: 3.3161742	total: 23.1ms	remaining: 2.54s
9:	learn: 3.2403968	total: 25.5ms	remaining: 2.53s
10:	learn: 3.1678202	total: 27.9ms	remaining: 2.51s
11:	learn: 3.0990047	total: 30.3ms	remaining: 2.49s
12:	learn: 3.0330467	total: 31.5ms	remaining: 2.39s
13:	learn: 2.9632860	total: 33.7ms	remaining: 2.38s
14:	learn: 2.8893200	total: 36.1ms	remaining: 2.37s
15:	learn: 2.8234467	total: 38.5ms	remaining: 2.37s
16:	learn: 2.7502017	total: 40.8ms	remaining: 2.36s
17:	learn: 2.6864924	total: 43.5ms	remaining: 2.37s
18:	learn: 2.6197992	total: 45.9ms	remaining:

214:	learn: 0.3399319	total: 571ms	remaining: 2.09s
215:	learn: 0.3389496	total: 575ms	remaining: 2.09s
216:	learn: 0.3378961	total: 578ms	remaining: 2.09s
217:	learn: 0.3369343	total: 581ms	remaining: 2.08s
218:	learn: 0.3356382	total: 584ms	remaining: 2.08s
219:	learn: 0.3341202	total: 587ms	remaining: 2.08s
220:	learn: 0.3334330	total: 590ms	remaining: 2.08s
221:	learn: 0.3325496	total: 593ms	remaining: 2.08s
222:	learn: 0.3310342	total: 597ms	remaining: 2.08s
223:	learn: 0.3295409	total: 601ms	remaining: 2.08s
224:	learn: 0.3282229	total: 606ms	remaining: 2.09s
225:	learn: 0.3270888	total: 609ms	remaining: 2.08s
226:	learn: 0.3248888	total: 611ms	remaining: 2.08s
227:	learn: 0.3241269	total: 615ms	remaining: 2.08s
228:	learn: 0.3233916	total: 618ms	remaining: 2.08s
229:	learn: 0.3222342	total: 621ms	remaining: 2.08s
230:	learn: 0.3201110	total: 623ms	remaining: 2.07s
231:	learn: 0.3185436	total: 625ms	remaining: 2.07s
232:	learn: 0.3179324	total: 628ms	remaining: 2.07s
233:	learn: 

440:	learn: 0.1605422	total: 1.13s	remaining: 1.43s
441:	learn: 0.1603876	total: 1.13s	remaining: 1.43s
442:	learn: 0.1601936	total: 1.13s	remaining: 1.43s
443:	learn: 0.1600580	total: 1.14s	remaining: 1.42s
444:	learn: 0.1595174	total: 1.14s	remaining: 1.42s
445:	learn: 0.1590867	total: 1.14s	remaining: 1.42s
446:	learn: 0.1589949	total: 1.15s	remaining: 1.42s
447:	learn: 0.1584174	total: 1.15s	remaining: 1.42s
448:	learn: 0.1576765	total: 1.15s	remaining: 1.42s
449:	learn: 0.1567237	total: 1.16s	remaining: 1.41s
450:	learn: 0.1560147	total: 1.16s	remaining: 1.41s
451:	learn: 0.1553439	total: 1.16s	remaining: 1.41s
452:	learn: 0.1551605	total: 1.16s	remaining: 1.4s
453:	learn: 0.1547139	total: 1.17s	remaining: 1.4s
454:	learn: 0.1537760	total: 1.17s	remaining: 1.4s
455:	learn: 0.1532284	total: 1.17s	remaining: 1.4s
456:	learn: 0.1527907	total: 1.17s	remaining: 1.39s
457:	learn: 0.1524047	total: 1.17s	remaining: 1.39s
458:	learn: 0.1517182	total: 1.18s	remaining: 1.39s
459:	learn: 0.15

663:	learn: 0.0820648	total: 1.7s	remaining: 861ms
664:	learn: 0.0818537	total: 1.71s	remaining: 859ms
665:	learn: 0.0815322	total: 1.71s	remaining: 857ms
666:	learn: 0.0812950	total: 1.71s	remaining: 854ms
667:	learn: 0.0809479	total: 1.71s	remaining: 852ms
668:	learn: 0.0806704	total: 1.72s	remaining: 850ms
669:	learn: 0.0806195	total: 1.72s	remaining: 848ms
670:	learn: 0.0803910	total: 1.72s	remaining: 845ms
671:	learn: 0.0801693	total: 1.73s	remaining: 843ms
672:	learn: 0.0800480	total: 1.73s	remaining: 841ms
673:	learn: 0.0799781	total: 1.73s	remaining: 838ms
674:	learn: 0.0797529	total: 1.74s	remaining: 836ms
675:	learn: 0.0796101	total: 1.74s	remaining: 833ms
676:	learn: 0.0793430	total: 1.74s	remaining: 831ms
677:	learn: 0.0790102	total: 1.75s	remaining: 829ms
678:	learn: 0.0789424	total: 1.76s	remaining: 831ms
679:	learn: 0.0788932	total: 1.76s	remaining: 830ms
680:	learn: 0.0787811	total: 1.77s	remaining: 827ms
681:	learn: 0.0784194	total: 1.77s	remaining: 825ms
682:	learn: 0

883:	learn: 0.0438843	total: 2.27s	remaining: 298ms
884:	learn: 0.0437921	total: 2.27s	remaining: 295ms
885:	learn: 0.0437409	total: 2.28s	remaining: 293ms
886:	learn: 0.0435888	total: 2.28s	remaining: 290ms
887:	learn: 0.0434031	total: 2.28s	remaining: 288ms
888:	learn: 0.0432752	total: 2.29s	remaining: 285ms
889:	learn: 0.0431324	total: 2.29s	remaining: 283ms
890:	learn: 0.0430267	total: 2.29s	remaining: 280ms
891:	learn: 0.0428197	total: 2.29s	remaining: 278ms
892:	learn: 0.0426904	total: 2.3s	remaining: 275ms
893:	learn: 0.0426504	total: 2.3s	remaining: 273ms
894:	learn: 0.0424581	total: 2.3s	remaining: 270ms
895:	learn: 0.0422977	total: 2.3s	remaining: 267ms
896:	learn: 0.0422199	total: 2.31s	remaining: 265ms
897:	learn: 0.0420842	total: 2.31s	remaining: 262ms
898:	learn: 0.0419751	total: 2.31s	remaining: 260ms
899:	learn: 0.0419074	total: 2.31s	remaining: 257ms
900:	learn: 0.0417626	total: 2.31s	remaining: 254ms
901:	learn: 0.0417161	total: 2.32s	remaining: 252ms
902:	learn: 0.04

In [87]:
ctb1 = CatBoostRegressor().fit(X1,r)
xgr1 = XGBRegressor().fit(X1,r)
lgr1 = LGBMRegressor().fit(X1,r)

Learning rate set to 0.034462
0:	learn: 19.7294539	total: 3.63ms	remaining: 3.62s
1:	learn: 19.2254477	total: 6.33ms	remaining: 3.16s
2:	learn: 18.7167513	total: 8.65ms	remaining: 2.88s
3:	learn: 18.2408508	total: 11ms	remaining: 2.74s
4:	learn: 17.7655829	total: 13.8ms	remaining: 2.74s
5:	learn: 17.3321028	total: 16.8ms	remaining: 2.78s
6:	learn: 16.8949587	total: 19.8ms	remaining: 2.8s
7:	learn: 16.4421269	total: 24.2ms	remaining: 3s
8:	learn: 16.0174561	total: 31.8ms	remaining: 3.5s
9:	learn: 15.5917353	total: 37.7ms	remaining: 3.73s
10:	learn: 15.2188485	total: 40.6ms	remaining: 3.65s
11:	learn: 14.8519783	total: 43ms	remaining: 3.54s
12:	learn: 14.5279477	total: 45.4ms	remaining: 3.44s
13:	learn: 14.1783423	total: 47.7ms	remaining: 3.36s
14:	learn: 13.8523749	total: 50ms	remaining: 3.29s
15:	learn: 13.5245290	total: 52.3ms	remaining: 3.22s
16:	learn: 13.1858809	total: 54.7ms	remaining: 3.16s
17:	learn: 12.8598447	total: 57ms	remaining: 3.11s
18:	learn: 12.5535890	total: 60.3ms	rem

212:	learn: 1.9013113	total: 563ms	remaining: 2.08s
213:	learn: 1.8896357	total: 567ms	remaining: 2.08s
214:	learn: 1.8837826	total: 570ms	remaining: 2.08s
215:	learn: 1.8739325	total: 573ms	remaining: 2.08s
216:	learn: 1.8671233	total: 576ms	remaining: 2.08s
217:	learn: 1.8603969	total: 579ms	remaining: 2.08s
218:	learn: 1.8493255	total: 582ms	remaining: 2.08s
219:	learn: 1.8465032	total: 585ms	remaining: 2.07s
220:	learn: 1.8424102	total: 589ms	remaining: 2.07s
221:	learn: 1.8380225	total: 592ms	remaining: 2.08s
222:	learn: 1.8346183	total: 596ms	remaining: 2.08s
223:	learn: 1.8307352	total: 599ms	remaining: 2.08s
224:	learn: 1.8276782	total: 601ms	remaining: 2.07s
225:	learn: 1.8206878	total: 604ms	remaining: 2.07s
226:	learn: 1.8118626	total: 606ms	remaining: 2.06s
227:	learn: 1.8082934	total: 609ms	remaining: 2.06s
228:	learn: 1.8041047	total: 611ms	remaining: 2.06s
229:	learn: 1.7995316	total: 614ms	remaining: 2.05s
230:	learn: 1.7961766	total: 616ms	remaining: 2.05s
231:	learn: 

438:	learn: 0.8662292	total: 1.14s	remaining: 1.45s
439:	learn: 0.8608923	total: 1.14s	remaining: 1.45s
440:	learn: 0.8579569	total: 1.15s	remaining: 1.45s
441:	learn: 0.8539813	total: 1.15s	remaining: 1.45s
442:	learn: 0.8522785	total: 1.15s	remaining: 1.45s
443:	learn: 0.8498079	total: 1.15s	remaining: 1.45s
444:	learn: 0.8455548	total: 1.16s	remaining: 1.44s
445:	learn: 0.8437876	total: 1.16s	remaining: 1.44s
446:	learn: 0.8420326	total: 1.16s	remaining: 1.44s
447:	learn: 0.8406812	total: 1.17s	remaining: 1.44s
448:	learn: 0.8371685	total: 1.17s	remaining: 1.44s
449:	learn: 0.8344796	total: 1.17s	remaining: 1.43s
450:	learn: 0.8334241	total: 1.17s	remaining: 1.43s
451:	learn: 0.8289603	total: 1.18s	remaining: 1.43s
452:	learn: 0.8256331	total: 1.18s	remaining: 1.42s
453:	learn: 0.8215079	total: 1.18s	remaining: 1.42s
454:	learn: 0.8170027	total: 1.18s	remaining: 1.42s
455:	learn: 0.8145864	total: 1.19s	remaining: 1.42s
456:	learn: 0.8112804	total: 1.19s	remaining: 1.41s
457:	learn: 

671:	learn: 0.4234155	total: 1.7s	remaining: 831ms
672:	learn: 0.4214920	total: 1.71s	remaining: 829ms
673:	learn: 0.4206674	total: 1.71s	remaining: 827ms
674:	learn: 0.4189943	total: 1.71s	remaining: 825ms
675:	learn: 0.4171763	total: 1.72s	remaining: 822ms
676:	learn: 0.4159098	total: 1.72s	remaining: 820ms
677:	learn: 0.4146961	total: 1.72s	remaining: 818ms
678:	learn: 0.4144273	total: 1.72s	remaining: 815ms
679:	learn: 0.4133480	total: 1.73s	remaining: 814ms
680:	learn: 0.4122355	total: 1.73s	remaining: 811ms
681:	learn: 0.4112457	total: 1.73s	remaining: 809ms
682:	learn: 0.4110434	total: 1.74s	remaining: 806ms
683:	learn: 0.4092682	total: 1.74s	remaining: 803ms
684:	learn: 0.4077735	total: 1.74s	remaining: 801ms
685:	learn: 0.4062552	total: 1.74s	remaining: 798ms
686:	learn: 0.4054240	total: 1.75s	remaining: 796ms
687:	learn: 0.4048958	total: 1.75s	remaining: 793ms
688:	learn: 0.4038590	total: 1.75s	remaining: 790ms
689:	learn: 0.4021765	total: 1.75s	remaining: 788ms
690:	learn: 0

829:	learn: 0.2654414	total: 2.08s	remaining: 427ms
830:	learn: 0.2648132	total: 2.09s	remaining: 425ms
831:	learn: 0.2642476	total: 2.09s	remaining: 422ms
832:	learn: 0.2637422	total: 2.09s	remaining: 420ms
833:	learn: 0.2632747	total: 2.1s	remaining: 417ms
834:	learn: 0.2621304	total: 2.1s	remaining: 415ms
835:	learn: 0.2612035	total: 2.1s	remaining: 413ms
836:	learn: 0.2608812	total: 2.11s	remaining: 410ms
837:	learn: 0.2606782	total: 2.11s	remaining: 408ms
838:	learn: 0.2598566	total: 2.11s	remaining: 405ms
839:	learn: 0.2587278	total: 2.11s	remaining: 403ms
840:	learn: 0.2583238	total: 2.12s	remaining: 400ms
841:	learn: 0.2579642	total: 2.12s	remaining: 398ms
842:	learn: 0.2571321	total: 2.12s	remaining: 395ms
843:	learn: 0.2562675	total: 2.12s	remaining: 392ms
844:	learn: 0.2556387	total: 2.13s	remaining: 390ms
845:	learn: 0.2553120	total: 2.13s	remaining: 387ms
846:	learn: 0.2547177	total: 2.13s	remaining: 385ms
847:	learn: 0.2537970	total: 2.13s	remaining: 382ms
848:	learn: 0.2

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000136 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 911
[LightGBM] [Info] Number of data points in the train set: 336, number of used features: 18
[LightGBM] [Info] Start training from score 68.202381
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

In [88]:
l = lgr.predict(X_)
c = ctb.predict(X_)
x = xgr.predict(X_)

l1 = lgr1.predict(X1_)
c1 = ctb1.predict(X1_)
x1 = xgr1.predict(X1_)

In [89]:
p=pd.DataFrame()
p['location_id']=X_test['location_id']
p['date']=X_test['date']
p['temperature_2m']=l
p['relative_humidity_2m']=l1
p

,location_id,date,temperature_2m,relative_humidity_2m
0,1,2024-09-24 00:00:00+00:00,17.358067,60.690656
1,1,2024-09-24 01:00:00+00:00,17.158225,64.611195
2,1,2024-09-24 02:00:00+00:00,17.107586,64.190433
3,1,2024-09-24 03:00:00+00:00,17.338445,65.012768
4,1,2024-09-24 04:00:00+00:00,17.269364,66.399847
...,...,...,...,...
163,1,2024-09-30 19:00:00+00:00,17.162127,83.545556
164,1,2024-09-30 20:00:00+00:00,17.310315,83.696854
165,1,2024-09-30 21:00:00+00:00,18.130379,87.454331
166,1,2024-09-30 22:00:00+00:00,17.735310,87.125500


In [90]:
p.to_csv('one_l.csv')